## Linking in Spark


<a target="_blank" href="https://colab.research.google.com/github/moj-analytical-services/splink/blob/splink4_dev/docs/demos/examples/spark/deduplicate_1k_synthetic.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


In [1]:
# Uncomment and run this cell if you're running in Google Colab.
# !pip install git+https://github.com/moj-analytical-services/splink.git@splink4_dev
# !pip install pyspark

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

from splink.backends.spark import similarity_jar_location

conf = SparkConf()
# This parallelism setting is only suitable for a small toy example
conf.set("spark.driver.memory", "12g")
conf.set("spark.default.parallelism", "8")
conf.set("spark.sql.codegen.wholeStage", "false")


# Add custom similarity functions, which are bundled with Splink
# documented here: https://github.com/moj-analytical-services/splink_scalaudfs
path = similarity_jar_location()
conf.set("spark.jars", path)

sc = SparkContext.getOrCreate(conf=conf)

spark = SparkSession(sc)
spark.sparkContext.setCheckpointDir("./tmp_checkpoints")

In [ ]:
# Disable warnings for pyspark - you don't need to include this
import warnings

spark.sparkContext.setLogLevel("ERROR")
warnings.simplefilter("ignore", UserWarning)

In [ ]:
from splink import splink_datasets

pandas_df = splink_datasets.fake_1000

df = spark.createDataFrame(pandas_df)

In [ ]:
import splink.comparison_library as cl
from splink import Linker, SettingsCreator, SparkAPI, block_on

settings = SettingsCreator(
    link_type="dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.NameComparison("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("first_name"),
        "l.surname = r.surname",  # alternatively, you can write BRs in their SQL form
    ],
    retain_intermediate_calculation_columns=True,
    em_convergence=0.01,
)

In [ ]:
linker = Linker(df, settings, database_api=SparkAPI(spark_session=spark))
deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    "l.email = r.email",
]

linker.training.estimate_probability_two_random_records_match(deterministic_rules, recall=0.6)

Probability two random records match is estimated to be  0.0806.                
This means that amongst all possible pairwise record comparisons, one in 12.41 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 40,246.67 matching pairs


In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=5e5)

----- Estimating u probabilities using random sampling -----
                                                                                
Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [ ]:
training_blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
training_session_fname_sname = (
    linker.training.estimate_parameters_using_expectation_maximisation(training_blocking_rule)
)

training_blocking_rule = "l.dob = r.dob"
training_session_dob = linker.training.estimate_parameters_using_expectation_maximisation(
    training_blocking_rule
)

                                                                                
----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name = r.first_name and l.surname = r.surname

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname
                                                                                
Iteration 1: Largest change in params was -0.706 in probability_two_random_records_match
Iteration 2: Largest change in params was 0.0583 in the m_probability of email, level `All other comparisons`
Iteration 3: Largest change in params was 0.0217 in the m_probability of email, level `All other comparisons`
Iteration 4: Largest change in params was 0.00941 in the m_probability of email, level `All other comparisons`

EM converged after

In [ ]:
results = linker.inference.predict(threshold_match_probability=0.9)

Blocking time: 1.18 seconds
24/07/08 21:03:19 ERROR CodeGenerator: failed to compile: org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 145, Column 16: Compiling "processNext()"
org.codehaus.commons.compiler.InternalCompilerException: Compiling "GeneratedClass" in File 'generated.java', Line 1, Column 1: File 'generated.java', Line 145, Column 16: Compiling "processNext()"
	at org.codehaus.janino.UnitCompiler.compile2(UnitCompiler.java:402)
	at org.codehaus.janino.UnitCompiler.access$000(UnitCompiler.java:236)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:363)
	at org.codehaus.janino.UnitCompiler$2.visitCompilationUnit(UnitCompiler.java:361)
	at org.codehaus.janino.Java$CompilationUnit.accept(Java.java:371)
	at org.codehaus.janino.UnitCompiler.compileUnit(UnitCompiler.java:361)
	at org.codehaus.janino.SimpleCompiler.cook(SimpleCompiler.java:264)
	at 

In [ ]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,tf_first_name_l,tf_first_name_r,bf_first_name,...,bf_city,bf_tf_adj_city,email_l,email_r,gamma_email,tf_email_l,tf_email_r,bf_email,bf_tf_adj_email,match_key
0,13.913529,0.999935,695,698,Noah,Noah,4,0.006,0.006,11.371023,...,0.626291,1.000000,noahh78@diaz-clark.com,noahh78@diaz-clark.com,4,0.005,0.005,8.473825,9.143944,0
1,8.439581,0.997128,331,333,Eleanor,Eleanor,4,0.004,0.004,11.371023,...,0.626291,1.000000,e.richards16@finley.ifo,e.richards16@finley.info,3,0.001,0.001,210.647668,1.000000,0
2,19.183862,0.999998,937,938,Isabelle,Isabelle,4,0.008,0.008,11.371023,...,0.626291,1.000000,isabelleh97m@lewis-gregory.com,isabelleh97@lewis-gregory.com,2,0.001,0.004,252.479846,1.000000,0
3,11.804017,0.999720,908,910,Isabelle,Isabelle,4,0.008,0.008,11.371023,...,0.626291,1.000000,isabellec@hill-williams.com,NaN,0,0.001,0.211,0.349824,1.000000,0
4,19.212145,0.999998,226,228,Julia,Julia,4,0.003,0.003,11.371023,...,5.870654,23.753086,julia.smith@english.org,julia.smith@english.org,4,0.002,0.002,8.473825,22.859860,0
